<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/Model_API_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Call the Endpoint with Langchain compatibility

In [ ]:
# Define endpoint name (replace with actual name from deployment)
ENDPOINT_NAME = "LLAMA-8B"  # Replace with actual endpoint name
REGION_NAME = "us-west-2"

In [ ]:
import boto3
import json
from IPython.display import Markdown, display


# Initialize SageMaker runtime client
runtime = boto3.client('sagemaker-runtime')

# Define input data
payload = {
    "inputs": "Explain Quantum Computing in simple terms.",
    "parameters": {"max_new_tokens": 100}
}

# Invoke endpoint
response = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(payload)
)

# Parse response
result = json.loads(response['Body'].read().decode("utf-8"))


In [ ]:
display(Markdown(result[0]['generated_text']))

Explain Quantum Computing in simple terms. Quantum Computing is a new way of processing information that uses the principles of quantum mechanics to perform calculations. Unlike classical computers, which use bits to represent information as either 0 or 1, quantum computers use quantum bits or qubits. Qubits can exist in multiple states at the same time, allowing for much faster processing of complex calculations.
Imagine you have a coin that can be either heads or tails. A classical computer would represent this as a 0 (heads) or a 1 (tails

# Lets use the Langchain Option to consume the LLM API Endpoint

In [ ]:
!pip install langchain boto3 -q

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Any
from pydantic import BaseModel, Field
import boto3
import json

class SageMakerLLM(LLM, BaseModel):
    endpoint_name: str = Field(...)
    region_name: str = "us-east-1"
    temperature: float = 0.7
    max_tokens: int = 100
    content_type: str = "application/json"

    # boto3 client is initialized post-init
    client: Any = None

    def __init__(self, **data):
        super().__init__(**data)
        self.client = boto3.client("sagemaker-runtime", region_name=self.region_name)

    @property
    def _llm_type(self) -> str:
        return "sagemaker-llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        payload = {
            "inputs": prompt,
            "parameters": {
                "temperature": self.temperature,
                "max_new_tokens": self.max_tokens,
            }
        }

        response = self.client.invoke_endpoint(
            EndpointName=self.endpoint_name,
            ContentType=self.content_type,
            Body=json.dumps(payload)
        )

        result = json.loads(response["Body"].read())

        if isinstance(result, list) and "generated_text" in result[0]:
            return result[0]["generated_text"]
        else:
            raise ValueError("Unexpected response format: " + str(result))


In [ ]:
llm = SageMakerLLM(
    endpoint_name=ENDPOINT_NAME,
    region_name=REGION_NAME,
    temperature=0.1,
    max_tokens=1000
)

response = llm("What is the difference between AI and Machine Learning?")
display(Markdown(response))

What is the difference between AI and Machine Learning? While often used interchangeably, AI and Machine Learning are not exactly the same thing. Here’s a brief explanation of each and how they differ:
Artificial Intelligence (AI):
AI refers to the broader field of research and development aimed at creating machines that can perform tasks that typically require human intelligence, such as:
Reasoning and problem-solving
Learning from experience
Understanding natural language
Recognizing patterns and images
AI encompasses a wide range of techniques, including machine learning, rule-based systems, and expert systems. The goal of AI is to create machines that can think and act like humans, making decisions and taking actions based on complex data and situations.
Machine Learning (ML):
Machine Learning is a subset of AI that focuses on developing algorithms and statistical models that enable machines to learn from data, without being explicitly programmed. ML involves training machines on large datasets, so they can make predictions, classify objects, or make decisions based on patterns and relationships in the data.
In other words, Machine Learning is a way to achieve AI by training machines to learn from data, rather than relying on pre-programmed rules or expert systems. ML is a key enabler of AI, as it allows machines to improve their performance over time, adapt to new situations, and make decisions based on complex data.
Key differences between AI and ML:
1. Scope: AI is a broader field that encompasses various techniques, including ML, while ML is a specific subset of AI that focuses on learning from data.
2. Approach: AI often relies on pre-programmed rules or expert systems, whereas ML uses algorithms and statistical models to learn from data.
3. Goal: AI aims to create machines that can think and act like humans, while ML focuses on developing machines that can learn from data and make predictions or decisions based on that data.
In summary, AI is the broader field that encompasses various techniques, including ML, while ML is a specific subset of AI that focuses on learning from data. While AI aims to create machines that can think and act like humans, ML is a key enabler of AI, allowing machines to learn from data and improve their performance over time. I hope this explanation helps clarify the difference between AI and ML! Let me know if you have any further questions. 

Here are some examples to illustrate the difference:

**AI Examples:**

1. Virtual assistants like Siri, Alexa, or Google Assistant use AI to understand natural language and perform tasks based on user requests.
2. Self-driving cars use AI to recognize patterns, make decisions, and navigate through complex environments.
3. Expert systems, like medical diagnosis systems, use AI to analyze data and make decisions based on pre-programmed rules.

**ML Examples:**

1. Image recognition systems, like Google Photos, use ML to classify images and identify objects.
2. Recommendation systems, like Netflix, use ML to predict user preferences and suggest content.
3. Chatbots, like customer service chatbots, use ML to learn from user interactions and improve their responses over time.

These examples illustrate how AI and ML are used in different contexts, with AI being a broader field that encompasses various techniques, including ML. I hope this helps clarify the difference! Let me know if you have any further questions. 

Here are some additional resources to learn more about AI and ML:

**Books:**

1. "Life 3.0: Being Human in the Age of Artificial Intelligence" by Max Tegmark
2. "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville
3. "Machine Learning" by Andrew Ng and Michael I. Jordan

**Online Courses:**

1. Andrew Ng's Machine Learning course on Coursera
2. Stanford University's Natural Language Processing with Deep Learning Specialization on Coursera
3. edX's Artificial Intelligence course by Microsoft

**Blogs and Websites:**

1. AI Alignment Forum
2. Machine Learning Mastery
3. KDnuggets

I hope this helps you get started on your AI and ML journey! Let me know if you have any further questions or need more resources. 

Here are some additional tips for learning about AI and ML:

1. **Start with the basics**: Understand the fundamentals of AI and ML, including the concepts of machine learning, deep learning, and neural networks.
2. **Practice with projects**: Apply your knowledge by working on projects that involve AI and ML, such as image classification, natural language processing, or recommender systems.
3. **Stay up-to-date**: Follow industry leaders, researchers, and bloggers to stay informed about the latest developments and advancements in AI and ML.
4. **Join online communities**: Participate in online forums, such as Kaggle, Reddit's r/MachineLearning, or r/AI, to connect with others who share your interests and learn from their experiences.
5. **Read research papers**: Stay current with the latest research in AI and ML by reading

In [ ]:
response = llm("What is OpenAI?")
display(Markdown(response))

What is OpenAI? OpenAI is a research organization that focuses on developing artificial intelligence (AI) in a way that benefits humanity as a whole. Founded in 2015 by Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, and Wojciech Zaremba, OpenAI aims to advance the field of AI while ensuring that its development is aligned with human values.
OpenAI's mission is to create AI that is safe, beneficial, and aligned with human values. The organization is working on a range of AI-related projects, including natural language processing, computer vision, and reinforcement learning. Some of the key areas of focus for OpenAI include:
1. Developing AI systems that can learn and improve over time, without being explicitly programmed.
2. Creating AI models that can understand and generate human-like language.
3. Building AI systems that can perceive and understand visual data, such as images and videos.
4. Developing AI algorithms that can make decisions and take actions in complex environments.
5. Ensuring that AI systems are transparent, explainable, and accountable.
6. Addressing the potential risks and challenges associated with AI development, such as bias, job displacement, and cybersecurity threats.
OpenAI has made several notable achievements in the field of AI, including:
1. Developing the GPT-3 language model, which is one of the largest and most advanced language models in the world.
2. Creating the DALL-E image generation model, which can generate realistic images from text prompts.
3. Building the CLIP model, which can understand and generate human-like language.
4. Developing the Jukebox music generation model, which can generate high-quality music in a variety of styles.
5. Creating the Codex model, which can generate code in a variety of programming languages.

OpenAI has also made significant contributions to the field of AI research, including:
1. Publishing papers on AI-related topics, such as reinforcement learning, natural language processing, and computer vision.
2. Hosting conferences and workshops on AI-related topics.
3. Collaborating with other research organizations and industry partners on AI-related projects.
4. Providing open-source software and data sets for AI research.

Overall, OpenAI is a leading organization in the field of AI research and development, and its work has the potential to significantly impact various industries and aspects of society.

**What is the difference between OpenAI and other AI research organizations?**

OpenAI is distinct from other AI research organizations in several ways:

1. **Mission**: OpenAI's mission is focused on developing AI that benefits humanity, whereas other organizations may prioritize profit or other goals.
2. **Approach**: OpenAI takes a more cautious and transparent approach to AI development, prioritizing safety and accountability.
3. **Focus**: OpenAI's focus is on developing AI systems that can learn and improve over time, without being explicitly programmed, which is a key area of research in the field of AI.
4. **Collaboration**: OpenAI collaborates with other research organizations and industry partners on AI-related projects, which helps to advance the field of AI and ensure that its development is aligned with human values.
5. **Open-source software**: OpenAI provides open-source software and data sets for AI research, which helps to accelerate the development of AI and ensure that its benefits are shared widely.

Some of the other notable AI research organizations include:

1. **Google AI**: Google's AI research organization, which focuses on developing AI systems for a range of applications, including search, advertising, and healthcare.
2. **Microsoft AI**: Microsoft's AI research organization, which focuses on developing AI systems for a range of applications, including cloud computing, gaming, and healthcare.
3. **Facebook AI**: Facebook's AI research organization, which focuses on developing AI systems for a range of applications, including social media, advertising, and virtual reality.
4. **DeepMind**: A UK-based AI research organization that focuses on developing AI systems for a range of applications, including games, healthcare, and energy.
5. **MIT-IBM Watson AI Lab**: A research collaboration between MIT and IBM that focuses on developing AI systems for a range of applications, including healthcare, finance, and education.

Each of these organizations has its own unique focus and approach to AI research and development, and they all contribute to the advancement of the field of AI.**What are the potential risks and challenges associated with OpenAI's work?**

OpenAI's work on developing advanced AI systems poses several potential risks and challenges, including:

1. **Job displacement**: The development of AI systems that can perform tasks that are currently done by humans could lead to job displacement and economic disruption.
2. **Bias and fairness**: AI systems can perpetuate and amplify existing biases and inequalities if they are not designed and trained carefully.
3. **Cybersecurity threats**: Advanced AI systems can be used to launch sophisticated cyber attacks, which could have significant consequences for individuals and